In [1]:
import sys

sys.path.append("/home/jovyan/work/sem-covid/")
sys.path = list(set(sys.path))

import os

os.getcwd()
os.chdir('/home/jovyan/work/sem-covid/')

import warnings

warnings.filterwarnings("ignore")

import numpy as np

from sem_covid.services.store_registry import store_registry

import pickle
import re
from typing import List, Tuple

import pandas as pd
import spacy

from gensim.parsing.preprocessing import remove_stopwords
from gensim.models import Word2Vec

In [3]:
LANGUAGE_MODEL = 'model1_language_model.model'
LANGUAGE_MODEL_MINIO_FOLDER = 'word2vec/'
LANGUAGE_MODEL_BUCKET_NAME = 'mdl-language'

In [4]:
minio = store_registry.minio_object_store(LANGUAGE_MODEL_BUCKET_NAME)

In [5]:
keyed_vector = pickle.loads(minio.get_object(LANGUAGE_MODEL_MINIO_FOLDER + LANGUAGE_MODEL))

In [12]:
words = keyed_vector.wv.index_to_key

In [13]:
len(words)

111356

In [24]:
filtered_words = [word for word in words if not word.isalpha()]

In [25]:
len(filtered_words)

94752

In [26]:
filtered_words

["'",
 ',',
 '.',
 ')',
 ':',
 '1',
 '19',
 '2',
 '5',
 '2020',
 '"',
 "'s",
 '3',
 '4',
 '[',
 ']',
 '000',
 '0',
 ';',
 '6',
 '2021',
 '/',
 '7',
 '8',
 '9',
 '14',
 '15',
 '25',
 '10',
 '24',
 '<',
 '>',
 'age_range',
 '12',
 '20',
 '30',
 '11',
 '2019',
 '13',
 '18',
 '34',
 '45',
 '35',
 '55',
 '100',
 '65',
 '44',
 '17',
 '64',
 '16',
 '31',
 '54',
 'social_partner',
 '75',
 'health_protection_surveillance_centre',
 '84',
 '85',
 '+',
 'positive_covid',
 'gender_patient',
 'hospital_statistic',
 '50',
 'department_health',
 '74',
 'statement_national_public_health_emergency_team',
 '22',
 '23',
 'event_person',
 '21',
 'young_people',
 '14_days',
 '28',
 '&',
 'today_s_case',
 '26',
 'community_transmission',
 '29',
 '500',
 '27',
 '19_ireland',
 'close_contact',
 '40',
 '2018',
 '60',
 "'cause",
 '19_community',
 'date_information_key_indicator',
 'chief_medical_officer',
 '200',
 '70',
 'minister_health',
 '48',
 'covid_19_ireland',
 'public_health_advice',
 '19_hasn_t_abroad_c

In [27]:
unified_df = store_registry.es_index_store().get_dataframe('ds_unified_datasets')

100% (4126 of 4126) |####################| Elapsed Time: 0:00:00 Time:  0:00:00


In [194]:
text = ' '.join(unified_df[['Title', 'Content']].agg(' '.join, axis=1).values)

In [ ]:
from nltk.corpus import words
import enchant

en_words = set(words.words())
d = enchant.Dict("en_US")

TypeError: 'Dict' object is not iterable

In [78]:
from sem_covid.adapters.embedding_models import BasicSentenceSplitterModel

splitter = BasicSentenceSplitterModel()

In [81]:
splitted_text = splitter.split(text)
len(splitted_text)

In [84]:
splitted_long_text = [sent for sent in splitted_text if len(sent) > 10]
len(splitted_long_text)

321504

In [199]:
def clean_text(text: str) -> str:
    tmp_text = re.sub(' +', ' ', re.sub(r'[^a-zA-Z]', ' ', text)).lower()
    tmp_text = remove_stopwords(tmp_text)
    tmp_words = tmp_text.split(' ')
    tmp_words = [word for word in tmp_words
                 if len(word) > 3 and (word in en_words or d.check(word))]
    if len(tmp_words) > 3:
        return ' '.join(tmp_words)
    else:
        return ''

In [200]:
cleaned_text = list(set([sent for sent in list(map(clean_text, splitted_long_text))
                         if sent]))

In [202]:
cleaned_text

218406

In [203]:
full_text = ' '.join(cleaned_text)

In [108]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vec = TfidfVectorizer()

In [204]:
transformed = tfidf_vec.fit_transform(raw_documents=cleaned_text)

In [205]:
tmp_words = list(tfidf_vec.vocabulary_.items())

In [210]:
len(tmp_words)

21045

In [126]:
def custom_cmp(item1, item2):
    return item1[1] - item2[1]

In [118]:
from functools import cmp_to_key

In [206]:
sorted(tmp_words, key=cmp_to_key(custom_cmp))





[('abac', 0),
 ('abandon', 1),
 ('abandoned', 2),
 ('abandoning', 3),
 ('abandonment', 4),
 ('abandons', 5),
 ('abate', 6),
 ('abated', 7),
 ('abatement', 8),
 ('abattoir', 9),
 ('abbey', 10),
 ('abbreviated', 11),
 ('abbreviation', 12),
 ('abbreviations', 13),
 ('abdal', 14),
 ('abdicated', 15),
 ('abdominal', 16),
 ('abeyance', 17),
 ('abhorrent', 18),
 ('abide', 19),
 ('abides', 20),
 ('abiding', 21),
 ('abigail', 22),
 ('abilities', 23),
 ('ability', 24),
 ('abiotic', 25),
 ('ablation', 26),
 ('able', 27),
 ('ably', 28),
 ('abnormal', 29),
 ('abnormally', 30),
 ('aboard', 31),
 ('abolish', 32),
 ('abolished', 33),
 ('abolishing', 34),
 ('abolishment', 35),
 ('abolition', 36),
 ('abort', 37),
 ('abortifacient', 38),
 ('abortion', 39),
 ('abortions', 40),
 ('abortus', 41),
 ('abrasive', 42),
 ('abrasiveness', 43),
 ('abreast', 44),
 ('abridged', 45),
 ('abroad', 46),
 ('abrogate', 47),
 ('abrogated', 48),
 ('abrupt', 49),
 ('abruptly', 50),
 ('absconding', 51),
 ('absence', 52),
 ('a